In [1]:
# Notebook for performing technical analysis and backtesting on Sharadar data (alpha)

# Note: this was originally written largely with the assistance of Gemini,
# although it needed a lot of extra prompting. I generated
# the grammar and reification modules by hand and extensively refactored the generated code.

In [2]:
import pandas as pd
import datetime
import typing
import unittest
import abc
import re
import typing
import inspect
import textwrap




In [3]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/INDICATORS.csv
/kaggle/input/SP500.csv
/kaggle/input/METRICS.csv
/kaggle/input/SF1.csv
/kaggle/input/SFP.csv
/kaggle/input/TICKERS.csv
/kaggle/input/SEP.csv
/kaggle/input/DAILY.csv
/kaggle/input/SF3B.csv
/kaggle/input/__notebook__.ipynb
/kaggle/input/SF3.csv
/kaggle/input/SF2.csv
/kaggle/input/ACTIONS.csv
/kaggle/input/EVENTS.csv
/kaggle/input/SF3A.csv


In [4]:
class Token:
    def __init__(self, type: str, value: str):
        self.type = type
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, '{self.value}')"

In [5]:
class ParseTreeNode:
    def __init__(self, type: str, value: typing.Optional[str] = None, children: typing.Optional[typing.List["ParseTreeNode"]] = None):
        self.type = type
        self.value = value
        self.children = children or []
        self.start_index = None
        self.end_index = None

    def __repr__(self):
        return f"ParseTreeNode({self.type}, value={self.value}, children={self.children}, start_index={self.start_index}, end_index={self.end_index})"
    def reify(self, function_factory):
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "+" and self.children[2].type == "term":
            return self.children[0].reify(function_factory) + self.children[2].reify(function_factory)
        # TODO: this isn't quite right
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "-" and self.children[2].type == "term":
            return self.children[0].reify(function_factory) - self.children[2].reify(function_factory)  
        if(self.type == "expression") and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "*" and self.children[2].type == "term":
            return self.children[0].reify(function_factory) * self.children[2].reify(function_factory)            
        # TODO: this isn't quite right
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "/" and self.children[2].type == "term":
            return self.children[0].reify(function_factory) / self.children[2].reify(function_factory)     
        if self.type == "term" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if self.type == "factor" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if self.type == "factor" and len(self.children) == 3 and self.children[0].value == "(" and self.children[2].value == ")":
            return self.children[1].reify(function_factory)
        if self.type == "number" and self.children is None or len(self.children) == 0:
            try:
                return int(self.value)
            except:
                return float(self.value)
        if self.type == "string" and self.children is None or len(self.children) == 0:
            return self.value
            
        if self.type == "factor" and len(self.children) == 4 and self.children[0].type == "identifier" and self.children[1].value == "(" and self.children[2].type == "arguments" and self.children[3].value == ")":

            identifier_node = self.children[0]
            name = identifier_node.value

            try:
                definition = function_factory.get(name)
            except ValueError:
                raise ValueError(f"Function '{name}' not found in the factory.")

            params = {}

            arguments_node = self.children[2]
            param_index = 0
            param_names = list(definition.parameters.keys())

            # Handle named arguments first
            named_params_processed = set()  # Keep track of named params

            for argument_node in arguments_node.children:
                if len(argument_node.children) == 3 and argument_node.children[1].type == "operator" and argument_node.children[1].value == "=":
                    param_name = argument_node.children[0].value
                    param_value_node = argument_node.children[2]

                    if param_name in named_params_processed: # Skip already processed named parameters
                        continue

                    try:
                        param_def = definition.parameters[param_name]
                    except KeyError:
                        raise ValueError(f"Parameter '{param_name}' not found for indicator '{name}'.")

                    param_value = param_value_node.reify(function_factory)  # Evaluate the value node
                    params[param_name] = param_value
                    named_params_processed.add(param_name) # Add to the set of processed named parameters

            # Next, handle positional arguments (skip named ones)
            for argument_node in arguments_node.children:
                if len(argument_node.children) == 1:  # Positional argument
                    try:
                        param_name = param_names[param_index]
                        if param_name in named_params_processed: # Skip if already named
                            param_index += 1
                            continue

                        param_def = definition.parameters[param_name]
                    except IndexError:
                        raise ValueError(f"Incorrect number of positional parameters for '{name}'.")

                    param_value_node = argument_node.children[0]
                    param_value = param_value_node.reify(function_factory)
                    params[param_name] = param_value
                    param_index += 1



            required_params = set(definition.parameters.keys())
            provided_params = set(params.keys())
            if required_params != provided_params:
                missing = required_params - provided_params
                raise ValueError(f"Missing required parameters for {name}: {missing}")

            return definition.create_function(**params)

        raise ValueError(f"Cannot reify node of type: {self.type} with {len(self.children)} children: {self}")



In [6]:
class GrammarRule:
    def __init__(self, left: str, right: typing.List[str]):
        self.left = left
        self.right = right

    def __repr__(self):
        return f"{self.left} -> {' '.join(self.right)}"



In [7]:
class Grammar:
    def __init__(self, grammar_string: str):
        """Initializes a Grammar object by parsing the grammar string."""
        self.rules = []
        for line in grammar_string.strip().splitlines():
            if line.strip():  # Skip empty lines
                parts = line.split("->")
                if len(parts) != 2:
                    raise ValueError(f"Invalid grammar rule: {line}")
                left = parts[0].strip()
                right = [part.strip() for part in parts[1].split()]
                self.rules.append(GrammarRule(left, right))  # Store rules as attributes


    def build_parse_tree(self, tokens: typing.List["Token"], start_symbol: str = "expression") -> typing.Optional["ParseTreeNode"]:
        """Builds a parse tree from a list of tokens using the grammar rules."""

        def _parse(index: int, nonterminal: str, current_depth=0) -> typing.Optional["ParseTreeNode"]:
            applicable_rules = [rule for rule in self.rules if rule.left == nonterminal]

            if index >= len(tokens):  # End of tokens
                if any(not rule.right for rule in applicable_rules): # Check for a matching epsilon rule
                    return ParseTreeNode(nonterminal, children=[])
                return None # No matching epsilon rule

            if not applicable_rules:
                return None

            for rule in applicable_rules:
                rule_matched = True
                children = []
                current_index = index

                for symbol in rule.right:
                    if current_index >= len(tokens):
                        rule_matched = False
                        break

                    if current_index < len(tokens):
                        token = tokens[current_index]

                        if (symbol == token.type) or (symbol == f'"{token.value}"') or \
                           (symbol == "identifier" and token.type == "identifier") or \
                           (symbol == "number" and token.type == "number") or \
                           (symbol == "string" and token.type == "string") or \
                           (symbol == "operator" and token.type == "operator"):
                            child = ParseTreeNode(token.type, value=token.value)
                            child.start_index = current_index
                            child.end_index = current_index
                            children.append(child)
                            current_index += 1  # Increment for terminal

                        elif any(gr.left == symbol for gr in self.rules):
                            child_node = _parse(current_index, symbol, current_depth + 1)
                            if child_node:
                                children.append(child_node)
                                current_index = child_node.end_index + 1
                            else:
                                rule_matched = False
                                break

                        else:
                            rule_matched = False
                            break

                if rule_matched:
                    node = ParseTreeNode(nonterminal, children=children)
                    node.start_index = children[0].start_index if children else index # Handle epsilon rules where children is empty
                    node.end_index = children[-1].end_index if children else index -1 # Handle epsilon rules where children is empty

                    return node

            return None

        return _parse(0, start_symbol)  # Allow specifying the start symbol
        
    def parse(self, input_string: str, start_symbol: str = "expression"):
        """Parses an input string into a parse tree."""
        tokens = self.tokenize(input_string)  # Tokenize the input string
        return self.build_parse_tree(tokens, start_symbol)

    def tokenize(self, expression: str) -> typing.List[Token]:
        """
        Tokenizes a string expression, splitting on spaces and identifying operators.
        """
    
        # Pattern to match tokens. Note: if we wanted to be really fancy, we would specify the token types in the grammar.
        pattern = r"(\*\*|\*|/|//|%|\+|-|==|!=|<=|>=|<|>|=|!|&&|\|\||&|\||\^|~|<<|>>|\(|\)|\[|\]|\{|\}|,|:|\.|->|@|=|;|\+=|-=|\*=|/=|//=|%=|&=|\|=|\^=|\<<=|>>=)|'([^']+)'|\"([^\"]+)\"|(\d+\.?\d*)|([a-zA-Z_]\w*)"
    
        tokens = []
        for match in re.finditer(pattern, expression):
            operator_match = match.group(1)
            single_quote_match = match.group(2)
            double_quote_match = match.group(3)
            number_match = match.group(4)
            identifier_match = match.group(5)
    
            if operator_match:
                tokens.append(Token("operator", operator_match))
            elif single_quote_match:
                tokens.append(Token("string", single_quote_match))
            elif double_quote_match:
                tokens.append(Token("string", double_quote_match))
            elif number_match:
                tokens.append(Token("number", number_match))
            elif identifier_match:
                tokens.append(Token("identifier", identifier_match))
            else:
                raise ValueError(f"invalid token in {expression}")
    
        return tokens

In [8]:
class FunctionInstance:
    def __init__(self, name: str, parameters: typing.Dict[str, typing.Any], definition):
        self.name = name
        self.parameters = parameters
        self.definition = definition

    def calculate(self, data: pd.DataFrame): 
        """
        Screens the data using the screener's definition and parameters.

        Args:
            data: The Pandas DataFrame containing the data.

        Returns:
            A Pandas Dataframe
        """
        return self.definition.calculate(data, self.parameters) # Pass the date to the definition

    def __repr__(self):
        params_str = ", ".join(f"{name}={value}" for name, value in self.parameters.items())
        return f"{self.definition.name}({params_str})"



In [9]:
class Parameter:
    """
    A class for specifying parameters for screeners and indicators.
    """

    def __init__(self,
                 name: str,
                 data_type: typing.Literal["integer", "real", "boolean", "string"],
                 min_val: typing.Union[int, float, None] = None,
                 max_val: typing.Union[int, float, None] = None,
                 default: typing.Any = None,
                 timeframe_defaults: typing.Dict[typing.Literal["tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"], typing.Any] = None,
                 increment: typing.Union[int, float, None] = None,
                 allowed_strings: typing.List[str] | None = None):
        if not isinstance(name, str):
            raise TypeError("name must be a string")
        if data_type not in ("integer", "real", "boolean", "string"):
            raise ValueError("data_type must be 'integer', 'real', 'boolean', or 'string'")

        if min_val is not None:
            if data_type == "integer" and not isinstance(min_val, int):
                raise TypeError("min_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(min_val, (int, float)):
                raise TypeError("min_val must be a number for real or integer data_type")

        if max_val is not None:
            if data_type == "integer" and not isinstance(max_val, int):
                raise TypeError("max_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(max_val, (int, float)):
                raise TypeError("max_val must be a number for real or integer data_type")

        if timeframe_defaults is not None:
            if not isinstance(timeframe_defaults, dict):
                raise TypeError("timeframe_defaults must be a dictionary")
            for timeframe in timeframe_defaults:
                if timeframe not in ("tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"):
                    raise ValueError(f"Invalid timeframe: {timeframe}")

        if data_type == "integer" and increment is None:
            increment = 1
        elif data_type == "real" and increment is None:
            increment = 0.01

        if data_type == "string" and allowed_strings is not None and not isinstance(allowed_strings, list):
          raise TypeError("allowed_strings must be a list of strings")

        if data_type != "string" and allowed_strings is not None:
          raise ValueError("allowed_strings can only be specified for string data type")


        self.name = name
        self.data_type = data_type
        self.min_val = min_val
        self.max_val = max_val
        self.default = default
        self.timeframe_defaults = timeframe_defaults or {}
        self.increment = increment
        self.allowed_strings = allowed_strings

    def get_default(self) -> typing.Any:
        return self.default

    def get_possible_values(self) -> typing.Iterable[typing.Any]:
        if self.data_type == "integer":
            if self.min_val is not None and self.max_val is not None:
                return range(self.min_val, self.max_val + 1)
        elif self.data_type == "real":
            if self.min_val is not None and self.max_val is not None:
                current = self.min_val
                while current <= self.max_val:
                    yield current
                    current += 0.01
        elif self.data_type == "boolean":
            return [True, False]
        elif self.data_type == "string":
            if self.allowed_strings is not None:  # Check if allowed_strings is defined
                return self.allowed_strings  # If defined, return those values
            else:
                return []  # Return an empty list if allowed_strings is None (unrestricted)
        return []

    def __repr__(self):
        return f"Parameter(name='{self.name}', data_type='{self.data_type}', min_val={self.min_val}, max_val={self.max_val}, default={self.default}, allowed_strings={self.allowed_strings})"






In [10]:
class FunctionDefinition:
    def __init__(self, name: str, parameters: typing.Dict[str, "Parameter"], calculation_function, factory=None): 
        if not isinstance(name, str):
            raise TypeError("name must be a string")

        if not isinstance(parameters, dict):
            raise TypeError("parameters must be a dictionary")

        if not all(isinstance(param, Parameter) for param in parameters.values()):
            raise TypeError("All values in parameters must be Parameter objects")

        if len(set(parameters.keys())) != len(parameters.keys()): # Check for duplicate keys
            raise ValueError("Parameter names must be unique.")

        if not callable(calculation_function):
            raise TypeError("calculation_function must be callable")

        self.name = name
        self.parameters = parameters
        self.calculation_function = calculation_function
        self.factory = factory

    def create_function(self, **kwargs: typing.Any) -> "FunctionInstance":
        params = {}
        for name, param_def in self.parameters.items():
            value = kwargs.get(name)

            if value is None:
                value = param_def.get_default()

            if param_def.data_type == "integer" and not isinstance(value, int):
                raise TypeError(f"Value for parameter '{name}' must be an integer")
            elif param_def.data_type == "real" and not isinstance(value, (int, float)):
                raise TypeError(f"Value for parameter '{name}' must be a number")
            elif param_def.data_type == "boolean" and not isinstance(value, bool):
                raise TypeError(f"Value for parameter '{name}' must be a boolean")
            elif param_def.data_type == "string" and not isinstance(value, str):
                raise TypeError(f"Value for parameter '{name}' must be a string")
            elif param_def.data_type in ("integer", "real"):
                if param_def.min_val is not None and value < param_def.min_val:  # Check min_val
                    raise ValueError(f"Value for parameter '{name}' must be greater than or equal to {param_def.min_val}")
                if param_def.max_val is not None and value > param_def.max_val:  # Check max_val
                    raise ValueError(f"Value for parameter '{name}' must be less than or equal to {param_def.max_val}")

            if param_def.data_type == "string" and param_def.allowed_strings is not None and value not in param_def.allowed_strings:
                raise ValueError(f"Value {value} is not in allowed strings for parameter {name}")

            params[name] = value

        return FunctionInstance(self.name, params, self)

    def calculate(self, data: pd.DataFrame, params: typing.Dict[str, typing.Any]) -> pd.DataFrame: # Type hint corrected
        """
        Calculates the function using the provided data and parameters.
        """
        kwargs = params.copy() 
        return self.calculation_function(data, **kwargs)

    def __repr__(self):
        return f"FunctionDefinition(name='{self.name}', parameters={self.parameters}, calculation_function={self.calculation_function.__name__ if hasattr(self.calculation_function, '__name__') else str(self.calculation_function)}, factory={self.factory})"


In [11]:
class FunctionFactory:
    """
    A class to manage a suite of function definitions.
    """

    def __init__(self):
        self.function_definitions: typing.Dict[str, Definition] = {}

    def register(self, function_definition):
        """
        Registers a new screener definition.

        Args:
            function_definition: The Definition to register.

        Raises:
            ValueError: If a screener with the same name is already registered.
        """
        if function_definition.name in self.function_definitions:
            raise ValueError(f"A screener with the name '{function_definition.name}' is already registered.")
        self.function_definitions[function_definition.name] = function_definition
        function_definition.ffactory = self

    def get(self, name: str):
        """
        Retrieves a function definition by name.

        Args:
            name: The name of the screener.

        Returns:
            The FunctionDefinition object.

        Raises:
            ValueError: If no screener with the given name is registered.
        """
        if name not in self.function_definitions:
            raise ValueError(f"No function found with the name '{name}'.")
        return self.function_definitions[name]

    def parse(self, indicator_string: str) -> FunctionInstance:
        """
        Parses an indicator string and generates an Function object.
        Supports both named and positional parameters.

        Args:
            indicator_string: A string representing the indicator and its parameters, 
                             e.g., "SMA(length=20)", "RSI(14)", "MACD(12, 26, 9)", "Bollinger Bands(20, 1.5)", "RVWAP(20)"

        Returns:
            A Function object.

        Raises:
            ValueError: If the indicator string is invalid or if required parameters are missing.
            TypeError: If the parameter values are of the wrong type.
        """

        try:
            name, params_str = indicator_string.split("(")  # Split name and parameters
            name = name.strip()
            params_str = params_str[:-1]  # Remove trailing ')'
        except ValueError:
            raise ValueError("Invalid indicator string format. Should be like: IndicatorName(param1=value1, param2=value2, ...) or IndicatorName(value1, value2, ...)")

        try:
            definition = self.get(name)  # Get the FunctionDefinition
        except ValueError:
            raise ValueError(f"Function '{name}' not found in the factory.")

        params = {}
        param_index = 0  # For positional parameters

        if params_str:
            for param_value_str in params_str.split(","):
                param_value_str = param_value_str.strip()

                try:
                    param_name, param_value = param_value_str.split("=")  # Try to split for named parameters
                    param_name = param_name.strip()
                    param_value = param_value.strip()

                    param_def = definition.parameters.get(param_name)
                    if not param_def:
                        raise ValueError(f"Parameter '{param_name}' not found for function '{name}'.")

                except ValueError:  # Positional parameter
                    param_name = list(definition.parameters.keys())[param_index]  # Get parameter name by index
                    param_def = definition.parameters[param_name] # Get the definition
                    param_value = param_value_str

                if param_def.data_type == "integer":
                    try:
                        param_value = int(param_value)
                    except ValueError:
                        raise TypeError(f"Parameter '{param_name}' must be an integer for function '{name}'.")
                elif param_def.data_type == "real":
                    try:
                        param_value = float(param_value)
                    except ValueError:
                        raise TypeError(f"Parameter '{param_name}' must be a float for function '{name}'.")
                elif param_def.data_type == "boolean":
                    param_value = param_value.lower() == "true"
                elif param_def.data_type == "string":
                    param_value = str(param_value)

                params[param_name] = param_value
                param_index += 1

        # Check if all required parameters are provided (both named and positional)
        required_params = set(definition.parameters.keys())
        provided_params = set(params.keys())
        if required_params != provided_params:
            missing = required_params - provided_params
            raise ValueError(f"Missing required parameters for {name}: {missing}")

        return definition.create_function(**params)

    def __repr__(self):
        return f"FunctionFactory(functions={self.function_definitions})"



In [12]:

# Example Screener Functions 

def top_n_screener_function(df, field, top_n):
    """
    Produces a boolean mask (pd.Series) for the top 5 readings per day.

    Args:
        df: Pandas DataFrame with columns for date and reading.
        date_col: Name of the column containing the date. Should be datetime or convertible.
        reading_col: Name of the column containing the reading.

    Returns:
        A pandas Series (boolean mask) with True for rows corresponding to the 
        top 5 readings for each day, and False otherwise. Returns
        an empty Series if the input DataFrame is empty.
    """
    foo = df.groupby("date")[field].rank(ascending=False, method='first')
    mask = foo <= top_n
    
    return mask


def percentile_screener_function(df, field, percentile):
    
    foo = df.groupby("date")[field].rank(ascending=False, method='first', pct=True)
    mask = foo >= percentile
    
    return mask



# Example Usage 
factory = FunctionFactory()

# Top N Screener FunctionDefinition and Registration
top_n_field_param = Parameter("field", "string", default="return")  # Example allowed strings
top_n_n_param = Parameter("top_n", "integer", min_val=1, default=5)
factory.register(FunctionDefinition("TopN", {"field": top_n_field_param, "top_n": top_n_n_param}, top_n_screener_function))

# Percentile Screener Definition and Registration
percentile_field_param = Parameter("field", "string", default="return")
percentile_percentile_param = Parameter("percentile", "real", min_val=0.0, max_val=1.0, default=.1)
factory.register(FunctionDefinition("Percentile", {"field": percentile_field_param, "percentile": percentile_percentile_param}, percentile_screener_function))


# Example DataFrame (replace with your data)
data = {'date': ['2024-01-01', '2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02', '2024-01-02'],
        'symbol': ['A', 'B', 'C', 'A', 'B', 'C'],
        'return': [0.10, 0.05, 0.15, 0.12, 0.08, 0.18],
        'other_field': [10, 20, 30, 15, 25, 35]}
df = pd.DataFrame(data)

# Create and use screeners
top_n_screener = factory.parse("TopN(top_n=2, field=return)")
top_n_result = top_n_screener.calculate(df)
print("Top N Result:\n", df.loc[top_n_result])

percentile_screener = factory.parse("Percentile(percentile=.5, field=return)")
percentile_result = percentile_screener.calculate(df)
print("Percentile Result:\n", df.loc[percentile_result])


Top N Result:
          date symbol  return  other_field
0  2024-01-01      A    0.10           10
2  2024-01-01      C    0.15           30
3  2024-01-02      A    0.12           15
5  2024-01-02      C    0.18           35
Percentile Result:
          date symbol  return  other_field
0  2024-01-01      A    0.10           10
1  2024-01-01      B    0.05           20
3  2024-01-02      A    0.12           15
4  2024-01-02      B    0.08           25


In [13]:
# Indicator Calculation Functions

def calculate_sma(df: pd.DataFrame, length: int) -> pd.DataFrame:
    print(f"calculate_sma(df, {length})")
    sma_values = df['close'].rolling(window=length).mean().values
    return pd.DataFrame({f"sma({length})": sma_values}, index=df.index)

def calculate_rsi(df: pd.DataFrame, length: int) -> pd.DataFrame:
    length = int(length)
    delta = df['close'].diff()
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    avg_gains = gains.rolling(window=length).mean()
    avg_losses = losses.rolling(window=length).mean()
    rs = avg_gains / avg_losses.replace(0, float('inf'))
    rsi = 100 - (100 / (1 + rs))
    rsi_values = rsi.values
    return pd.DataFrame({f"rsi({length})": rsi_values}, index=df.index)

def calculate_macd(df: pd.DataFrame, fast_length: int, slow_length: int, signal_length: int) -> pd.DataFrame:
    ema_fast = df['close'].ewm(span=fast_length, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow_length, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_length, adjust=False).mean()
    macd_values = macd.values
    return pd.DataFrame({f"macd({fast_length},{slow_length},{signal_length})": macd_values}, index=df.index)  # No alignment needed for MACD

def calculate_bollinger_bands(df: pd.DataFrame, length: int, std_dev: float) -> pd.DataFrame:
    rolling_mean = df['close'].rolling(window=length).mean()
    rolling_std = df['close'].rolling(window=length).std()
    upper_band = rolling_mean + (rolling_std * std_dev)
    lower_band = rolling_mean - (rolling_std * std_dev)
    middle_values = rolling_mean.values
    upper_values = upper_band.values
    lower_values = lower_band.values
    bb_df = pd.DataFrame({f'bb({length},{std_dev})_middle': middle_values, f'bb({length},{std_dev})_upper': upper_values, f'bb({length},{std_dev})_lower': lower_values}, index=df.index)
    return bb_df

def calculate_rvwap(df: pd.DataFrame, length: int) -> pd.DataFrame:
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    rolling_volume = df['volume'].rolling(length).sum()
    typical_price_x_volume = df["volume"] * typical_price
    rolling_typical_price_x_volume = typical_price_x_volume.rolling(length).sum()
    vwap = rolling_typical_price_x_volume / rolling_volume
    return pd.DataFrame({f"rvwap({length})": vwap.values}, index=df.index)

# Example usage

# SMA
sma_length_param = Parameter("length", "integer", min_val=1, max_val=200, default=20)
factory.register(FunctionDefinition("SMA", {"length": sma_length_param}, calculate_sma))

# RSI
rsi_length_param = Parameter("length", "integer", min_val=1, max_val=200, default=14) # Different default length
factory.register(FunctionDefinition("RSI", {"length": rsi_length_param}, calculate_rsi))

# MACD
fast_length_param = Parameter("fast_length", "integer", min_val=1, max_val=100, default=12)
slow_length_param = Parameter("slow_length", "integer", min_val=1, max_val=200, default=26)
signal_length_param = Parameter("signal_length", "integer", min_val=1, max_val=50, default=9)
factory.register(FunctionDefinition("MACD", {"fast_length": fast_length_param, "slow_length": slow_length_param, "signal_length": signal_length_param}, calculate_macd))

# Bollinger Bands
bb_length_param = Parameter("length", "integer", min_val=1, max_val=200, default=20)
std_dev_param = Parameter("std_dev", "real", min_val=0.1, max_val=5.0, default=2.0, increment=0.1)
factory.register(FunctionDefinition("Bollinger Bands", {"length": bb_length_param, "std_dev": std_dev_param}, calculate_bollinger_bands))

# VWAP
vwap_length_param = Parameter("length", "integer", min_val=1, max_val=200, default=20)
factory.register(FunctionDefinition("RVWAP", {"length": vwap_length_param}, calculate_rvwap))


data = {
    'date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10',
             '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-20',
             '2024-01-21', '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-27', '2024-01-28', '2024-01-29', '2024-01-30'],
    'symbol': ['ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC',
               'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC',
               'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC'],
    'close': [150.25, 152.75, 151.50, 153.00, 154.25, 153.75, 155.00, 156.50, 155.25, 157.00,
              158.25, 159.75, 158.50, 160.00, 161.25, 160.75, 162.00, 163.50, 162.25, 164.00,
              165.25, 166.75, 165.50, 167.00, 168.25, 167.75, 169.00, 170.50, 169.25, 171.00],
    'volume': [100000, 120000, 80000, 90000, 110000, 100000, 150000, 160000, 130000, 110000,
               120000, 140000, 90000, 100000, 130000, 110000, 160000, 170000, 140000, 120000,
               130000, 150000, 100000, 110000, 140000, 120000, 170000, 180000, 150000, 130000],
    'high': [152.50, 155.00, 153.75, 155.25, 156.50, 156.00, 157.25, 158.75, 157.50, 159.25,
             160.50, 162.00, 160.75, 162.25, 163.50, 163.00, 164.25, 165.75, 165.50, 167.25,
             168.50, 170.00, 168.75, 170.25, 171.50, 171.00, 172.25, 173.75, 173.50, 175.25],
    'low': [148.00, 150.50, 149.25, 150.75, 152.00, 151.50, 152.75, 154.25, 153.00, 154.75,
            156.00, 157.50, 156.25, 157.75, 159.00, 158.50, 159.75, 161.25, 160.00, 161.75,
            163.00, 164.50, 163.25, 164.75, 166.00, 165.50, 166.75, 168.25, 167.00, 168.75]
}

df = pd.DataFrame(data)  # No index

# Calculate indicators

sma_indicator = factory.parse("SMA(5)")  # Or SMA(length=5)
print(sma_indicator)
sma_result = sma_indicator.calculate(df)
df = df.join(sma_result)  # Add the result to your DataFrame

rsi_indicator = factory.parse("RSI(14)")  # Or RSI(length=14)
rsi_result = rsi_indicator.calculate(df)
df = df.join(rsi_result)

macd_indicator = factory.parse("MACD(12, 26, 9)")  # Or MACD(fast_length=12, slow_length=26, signal_length=9)
macd_result = macd_indicator.calculate(df)
df = df.join(macd_result)

rvwap_indicator = factory.parse("RVWAP(20)") # Or RVWAP(length=20)
rvwap_result = rvwap_indicator.calculate(df)
df = df.join(rvwap_result)

print(sma_indicator)
print(rsi_indicator)
print(macd_indicator)
print(rvwap_indicator)

print(df)

####

SMA(length=5)
calculate_sma(df, 5)
SMA(length=5)
RSI(length=14)
MACD(fast_length=12, slow_length=26, signal_length=9)
RVWAP(length=20)
          date symbol   close  volume    high     low  sma(5)    rsi(14)  \
0   2024-01-01    ABC  150.25  100000  152.50  148.00     NaN        NaN   
1   2024-01-02    ABC  152.75  120000  155.00  150.50     NaN        NaN   
2   2024-01-03    ABC  151.50   80000  153.75  149.25     NaN        NaN   
3   2024-01-04    ABC  153.00   90000  155.25  150.75     NaN        NaN   
4   2024-01-05    ABC  154.25  110000  156.50  152.00  152.35        NaN   
5   2024-01-06    ABC  153.75  100000  156.00  151.50  153.05        NaN   
6   2024-01-07    ABC  155.00  150000  157.25  152.75  153.50        NaN   
7   2024-01-08    ABC  156.50  160000  158.75  154.25  154.50        NaN   
8   2024-01-09    ABC  155.25  130000  157.50  153.00  154.95        NaN   
9   2024-01-10    ABC  157.00  110000  159.25  154.75  155.50        NaN   
10  2024-01-11    ABC  158.25

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [14]:
# FIXME: the grammar and reification modules do not correctly handle order of operations correctly for operations
# that do not have the associative property. An easy workaround is to require parentheses.
# The parser does not handle epsilon correctly either
grammar_string = """
expression -> term "+" term
expression -> term "-" term
expression -> term "*" term
expression -> term "/" term
expression -> term
term -> factor
factor -> "(" expression ")"
factor -> number
factor -> string
factor -> identifier "(" arguments ")"
factor -> identifier
arguments -> argument "," arguments
arguments -> argument
argument -> identifier "=" expression
argument -> expression
"""

grammar = Grammar(grammar_string)
print("Parsed Grammar:")
print(grammar)

expression = "AAPL + MSFT * GOOG / 2.5"
parse_tree = grammar.parse(expression)
print("\nParse Tree:")
print(expression)
print(parse_tree)

expression2 = " 'hello world' + \"another string\" - 123.45 * (variable_name / 2) "
parse_tree2 = grammar.parse(expression2)
print("\nParse Tree 2:")
print(expression2)
print(parse_tree2)

expression3 = "TopN(top_n=2, field='return')" # Test for nested functions
parse_tree3 = grammar.parse(expression3)
print("\nParse Tree 3:")
print(expression3)
print(parse_tree3)

expression4 = "TopN(top_n=2, field=return)" # Test for nested functions without quotes
parse_tree4 = grammar.parse(expression4)
print("\nParse Tree 4:")
print(expression4)
print(parse_tree4)

expression5 = "AAPL+MSFT*GOOG/2.5>100&&(RSI(14)<30||close=='test')"  # No spaces around operators
parse_tree5 = grammar.parse(expression5)
print("\nParse Tree 5:")
print(expression5)
print(parse_tree5)

print("\nParse Tree a:")
expression = "(10 - 1) - 1"
print(expression)
parse_tree = grammar.parse(expression)
print(parse_tree)
reified_expression = parse_tree.reify(factory)  # Call reify as a method
print(reified_expression)

# ... (Other examples – call reify as a method on the parse tree)
print("\nParse Tree b:")
expression = "SMA(length=10)"
print(expression)
parse_tree = grammar.parse(expression)
print(parse_tree)
reified_expression = parse_tree.reify(factory)
print(reified_expression)

expression = "10"
print("\nParse Tree c:")
print(expression)
parse_tree = grammar.parse(expression)
print(parse_tree)
reified_expression = parse_tree.reify(factory)
print(reified_expression)



Parsed Grammar:

Parse Tree:
AAPL + MSFT * GOOG / 2.5
ParseTreeNode(expression, value=None, children=[ParseTreeNode(term, value=None, children=[ParseTreeNode(factor, value=None, children=[ParseTreeNode(identifier, value=AAPL, children=[], start_index=0, end_index=0)], start_index=0, end_index=0)], start_index=0, end_index=0), ParseTreeNode(operator, value=+, children=[], start_index=1, end_index=1), ParseTreeNode(term, value=None, children=[ParseTreeNode(factor, value=None, children=[ParseTreeNode(identifier, value=MSFT, children=[], start_index=2, end_index=2)], start_index=2, end_index=2)], start_index=2, end_index=2)], start_index=0, end_index=2)

Parse Tree 2:
 'hello world' + "another string" - 123.45 * (variable_name / 2) 
ParseTreeNode(expression, value=None, children=[ParseTreeNode(term, value=None, children=[ParseTreeNode(factor, value=None, children=[ParseTreeNode(string, value=hello world, children=[], start_index=0, end_index=0)], start_index=0, end_index=0)], start_index=0

In [15]:
# This code is used for prompt engineering

def generate_simplified_signature_comment(obj):
    """Generates commented signatures and stubs, excluding unhelpful docstrings (1-space indent)."""

    comment = ""

    if inspect.isclass(obj):
        comment += f"# class {obj.__name__}:\n"

        class_docstring = inspect.getdoc(obj)
        if class_docstring and not _is_unhelpful_docstring(class_docstring):  # Check docstring
            wrapped_class_docstring = textwrap.dedent(class_docstring).strip()
            comment += f"#  \"\"\"" + "\n"  # 1-space indent
            for line in wrapped_class_docstring.splitlines():
                comment += f"#   {line}\n"  # 1-space indent + 1 space for line content
            comment += f"#  \"\"\"" + "\n"  # 1-space indent

        members = inspect.getmembers(obj)
    elif inspect.isfunction(obj):
        comment += f"# def {obj.__name__}:\n"
        members = [(obj.__name__, obj)]
    else:
        return None

    for name, member in members:
        if inspect.isfunction(member) or inspect.ismethod(member):
            signature = inspect.signature(member)
            params = []
            param_defs = []
            for param in signature.parameters.values():
                param_type = typing.get_type_hints(member).get(param.name) or "Any"
                default = ""
                if param.default is not inspect.Parameter.empty:
                    default = f" = {param.default!r}"
                params.append(f"{param.name}: {param_type}{default}")
                param_defs.append(f"{param.name}{default}")

            return_type = typing.get_type_hints(member).get('return') or "Any"

            comment += f"#  def {name}({', '.join(param_defs)}):\n"  # 1-space indent
            comment += f"#   # -> {return_type}\n"  # 1-space indent + 1 space for line content

            docstring = inspect.getdoc(member)
            if docstring and not _is_unhelpful_docstring(docstring):  # Check docstring
                wrapped_docstring = textwrap.dedent(docstring).strip()
                comment += f"#   \"\"\"" + "\n"  # 1-space indent
                for line in wrapped_docstring.splitlines():
                    comment += f"#    {line}\n"  # 1-space indent + 2 spaces for line content
                comment += f"#   \"\"\"" + "\n"  # 1-space indent
            comment += f"#   pass\n"  # 1-space indent

    return comment


def _is_unhelpful_docstring(docstring):
    """Checks if a docstring is likely to be auto-generated (optimized string search)."""

    unhelpful_phrases = [
        "initialize self.  see help(type(self)) for accurate signature.",
        "see help(type(self)) for accurate signature.",
        "method generated by ide",
        "return repr(self)",
    ]

    cleaned_docstring = docstring.strip().lower()  # Lowercase and strip ONCE

    for phrase in unhelpful_phrases:
        if phrase in cleaned_docstring:  # Use simple string containment check
            return True

def _get_fully_qualified_type_name(type_hint):
    if type_hint is None:
        return "None"

    origin = typing.get_origin(type_hint)  # Use typing.get_origin

    if origin is not None:  # Generic type (List, Dict, etc.)
        args = typing.get_args(type_hint)   # Use typing.get_args
        if origin is typing.List:         # Use typing.List
            arg_str = ", ".join(_get_fully_qualified_type_name(arg) for arg in args) if args else ""
            return f"typing.List[{arg_str}]"
        elif origin is typing.Dict:        # Use typing.Dict
            arg_str = ", ".join(_get_fully_qualified_type_name(arg) for arg in args) if args else ""
            return f"typing.Dict[{arg_str}]"
        elif origin is typing.Optional:    # Use typing.Optional
            arg_str = _get_fully_qualified_type_name(args[0]) if args else ""
            return f"typing.Optional[{arg_str}]"
        elif origin is typing.Tuple:       # Use typing.Tuple
            arg_str = ", ".join(_get_fully_qualified_type_name(arg) for arg in args) if args else ""
            return f"typing.Tuple[{arg_str}]"
        elif origin is typing.Union:       # Use typing.Union. Added support for Union
            arg_str = ", ".join(_get_fully_qualified_type_name(arg) for arg in args) if args else ""
            return f"typing.Union[{arg_str}]"
        else:
            return origin.__module__ + "." + origin.__name__ if hasattr(origin, '__module__') else origin.__name__  # Handle other generics
    elif hasattr(type_hint, '__module__') and hasattr(type_hint, '__name__'):  # Regular class
        return type_hint.__module__ + "." + type_hint.__name__
    elif hasattr(type_hint, '__name__'):  # Regular class
        return type_hint.__name__
    else:
        return str(type_hint)  # Fallback to string representation

def list_module_objects(module=None):
    """Lists all functions and classes defined in the current module.

    Args:
        module: The module to inspect. If None, defaults to the current module.

    Returns:
        A list of tuples, where each tuple contains the name and the object 
        (function or class).  Returns an empty list if no suitable objects are found.
    """

    if module is None:
        import sys
        module = sys.modules[__name__]  # Get the current module

    objects = []
    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj) or inspect.isclass(obj):
            if obj.__module__ == module.__name__: #check if object is defined in the current module
                objects.append((name, obj))
    return objects


if __name__ == "__main__":
    # Example usage:
    module_objects = list_module_objects()

    for name, obj in module_objects:
        comment = generate_simplified_signature_comment(obj)
        if comment:
            print(comment, end="#\n")
        #if you want to execute the prompt engineering on the current module
        #and write the output to a file, you can do this:
        # with open("prompt_engineering_output.txt", "a") as f:
        #     f.write(comment + "\n")

    # To list objects from another module (if needed):
    # import my_other_module
    # other_module_objects = list_module_objects(my_other_module)
    # for name, obj in other_module_objects:
    #   # ... process objects from the other module

# class FunctionDefinition:
#  def __init__(self, name, parameters, calculation_function, factory = None):
#   # -> Any
#   pass
#  def __repr__(self):
#   # -> Any
#   pass
#  def calculate(self, data, params):
#   # -> <class 'pandas.core.frame.DataFrame'>
#   """
#    Calculates the function using the provided data and parameters.
#   """
#   pass
#  def create_function(self, kwargs):
#   # -> <class '__main__.FunctionInstance'>
#   pass
#
# class FunctionFactory:
#  """
#   A class to manage a suite of function definitions.
#  """
#  def __init__(self):
#   # -> Any
#   pass
#  def __repr__(self):
#   # -> Any
#   pass
#  def get(self, name):
#   # -> Any
#   """
#    Retrieves a function definition by name.
#    
#    Args:
#        name: The name of the screener.
#    
#    Returns:
#        The FunctionDefinition object.
#    
#    Raises:
#        ValueError: If no screener with the given name is registered.
#   """
#   pass
#  def parse(self, indicator_string):
#   # -> <class